In [1]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

In [2]:
df_SCC = pd.read_excel('1-s2.0-S0950061820339581-mmc1.xlsx', 'Table A1')
df_chem = pd.read_excel('1-s2.0-S0950061820339581-mmc1.xlsx', 'Table A2')

In [3]:
df_SCC.shape

(1588, 146)

In [4]:
def del_all_missing_col(df):
    cols_w_missing = list(df.columns[df.isnull().all()])
    cols = list(df.columns)

    for col in cols_w_missing:
        cols.remove(col)
    return df[cols]

In [5]:
df_SCC = del_all_missing_col(df_SCC)
df_chem = del_all_missing_col(df_chem)

In [6]:
# set up the hierarchical index of df_SCC_strength
index_level_0 = df_SCC.iloc[11:, 1].to_numpy().astype(int)
column_level_0 = df_SCC.iloc[9, 2:].fillna(method='ffill').to_numpy()
column_level_1 = df_SCC.iloc[10, 2:].fillna(method='ffill').to_numpy()

index_arrays = [index_level_0]
column_arrays = [column_level_0, column_level_1]

all_values = df_SCC.iloc[11:, 2:].values

df_SCC_strength = pd.DataFrame(all_values, index=index_arrays, columns=column_arrays)

# drop the record without strength information
df_SCC_strength_cyld = df_SCC_strength["f'c-cylinder (MPa)"]['28-d']
df_SCC_strength_cube = df_SCC_strength["f'c-cube  (MPa)"]['28-d']
df_SCC_strength_null = df_SCC_strength_cyld.combine_first(df_SCC_strength_cube).isnull().values
df_SCC_strength_notnull = df_SCC_strength_cyld.combine_first(df_SCC_strength_cube*0.8).dropna()

df_SCC_strength = df_SCC_strength[~df_SCC_strength_null]

In [7]:
scc_fresh = df_SCC_strength['Fresh Properties of SCCs']['Slump flow (mm)']
scc_fresh = scc_fresh.fillna(scc_fresh.median()) # fill the missing value with median

In [8]:
scc_mixing_ratio = df_SCC_strength['Mixing ratio'] # no missing values

In [9]:
# create scc_mix_proportion
scc_mix_proportion = df_SCC_strength['Mix Proportion (kg/m3)']

# re-format scc_mix_proportion

# re-format the colums related to PM1 and PM2 in scc_mix_proportion
scc_mix_proportion_other_PM_cols = scc_mix_proportion.columns[5:9]

scc_mix_proportion_PM1 = pd.get_dummies(scc_mix_proportion[scc_mix_proportion_other_PM_cols]['Type 1 PM'])
scc_mix_proportion_PM1 = scc_mix_proportion_PM1.mul(scc_mix_proportion['Other PM 1'], axis=0)
scc_mix_proportion_PM1.fillna(0, inplace=True)

scc_mix_proportion_PM2 = pd.get_dummies(scc_mix_proportion[scc_mix_proportion_other_PM_cols]['Type 2 PM'])
scc_mix_proportion_PM2 = scc_mix_proportion_PM2.mul(scc_mix_proportion['Other PM 2'], axis=0)
scc_mix_proportion_PM2.fillna(0, inplace=True)

scc_mix_proportion_PM1_pumice = scc_mix_proportion_PM2['Pumice'].combine_first(scc_mix_proportion_PM1['Pumice powder']).to_frame()

scc_mix_proportion_PM1 = scc_mix_proportion_PM1.drop('Pumice powder', axis=1)

scc_mix_proportion_PM = pd.concat([scc_mix_proportion_PM1, scc_mix_proportion_PM1_pumice], axis=1)


In [10]:

# scc_mix_proportion_F_cols = scc_mix_proportion.columns[5:9]

# scc_mix_proportion_F_cols = scc_mix_proportion.columns[9:15]

# scc_mix_proportion_F1 = pd.get_dummies(scc_mix_proportion[scc_mix_proportion_F_cols]['Type1 F'])
# scc_mix_proportion_F1 = scc_mix_proportion_F1.mul(scc_mix_proportion['Filler1'], axis=0)
# scc_mix_proportion_F1.fillna(0, inplace=True)

# scc_mix_proportion_F1

# scc_mix_proportion_F2 = pd.get_dummies(scc_mix_proportion[scc_mix_proportion_F_cols]['Type2 F'])
# scc_mix_proportion_F2 = scc_mix_proportion_F2.mul(scc_mix_proportion['Filler2'], axis=0)
# scc_mix_proportion_F2.fillna(0, inplace=True)

# scc_mix_proportion_F2

# scc_mix_proportion_F3 = pd.get_dummies(scc_mix_proportion[scc_mix_proportion_F_cols]['Type3 F'])
# scc_mix_proportion_F3 = scc_mix_proportion_F3.mul(scc_mix_proportion['Filler3'], axis=0)
# scc_mix_proportion_F3.fillna(0, inplace=True)
# scc_mix_proportion_F3

In [11]:

# rearrange the order of the columns of scc_mix_proportion
scc_mix_proportion = pd.concat([scc_mix_proportion.iloc[:,:5], scc_mix_proportion_PM, scc_mix_proportion.iloc[:,-5:]], axis=1).fillna(0)

# add source and year columns to scc_mix_proportion
scc_source_year = pd.DataFrame(df_SCC_strength.loc[:, ['Source', 'Year']].values, index=df_SCC_strength.index, columns=['Source', 'Year'])
scc_mix_proportion = pd.concat([scc_source_year, scc_mix_proportion], axis=1)

# chang year column to string type
scc_mix_proportion['Year'] = scc_mix_proportion['Year'].astype(str)

# rename some columns for future cleaning
scc_mix_proportion = scc_mix_proportion.rename(columns={'Cement':'OPC', 'ASH':'FA', 'Kaolin':'MK'})


In [12]:
scc_strength_clean = pd.concat([scc_mix_proportion, scc_mixing_ratio, scc_fresh, df_SCC_strength_notnull], axis=1)


In [13]:
scc_mix_proportion_source_year = scc_strength_clean[['Source','Year']]

scc_mix_proportion_source_year_w_issue = [scc_mix_proportion_source_year.values[16], scc_mix_proportion_source_year.values[305]\
                                         , scc_mix_proportion_source_year.values[616], scc_mix_proportion_source_year.values[900]\
                                         , scc_mix_proportion_source_year.values[932], scc_mix_proportion_source_year.values[990]]

deter1 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[0])

deter2 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[1])

deter3 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[2])

deter4 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[3])

deter5 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[4])

deter6 = (scc_strength_clean[['Source','Year']].values == scc_mix_proportion_source_year_w_issue[5])


index = deter1[:, 0] & deter1[:, 1] | deter2[:, 0] & deter2[:, 1] | deter3[:, 0] & deter3[:, 1]|\
        deter4[:, 0] & deter4[:, 1] | deter5[:, 0] & deter5[:, 1] | deter6[:, 0] & deter6[:, 1]

scc_strength_clean_w_issue = scc_strength_clean[index]

scc_strength_clean_w_issue


,Source,Year,OPC,FA,GGBFS,SF,MK,BA,CKD,NP,...,Water,Added Water,Superplasticizer,water -eff,w/b (eff),w/b (total),b/CA,b/FA,Slump flow (mm),28-d
15,Bui et al.,2002,350.0,162.0,0.0,0.0,0.0,0.0,0.0,0.0,...,208.0,0,1.940,209.16400,0.406250,0.408523,0.609524,0.666667,670.000,41.36000
19,Bui et al.,2002,349.0,162.0,0.0,0.0,0.0,0.0,0.0,0.0,...,199.0,0,3.130,200.87800,0.389432,0.393108,0.599765,0.655969,762.000,47.92000
24,Bui et al.,2002,350.0,133.0,0.0,0.0,0.0,0.0,0.0,0.0,...,184.0,0,3.620,186.17200,0.380952,0.385449,0.546999,0.592638,724.000,44.24000
29,Bui et al.,2002,350.0,111.0,0.0,0.0,0.0,0.0,0.0,0.0,...,180.0,0,3.470,182.08200,0.390456,0.394972,0.512222,0.554753,711.000,48.80000
31,Bui et al.,2002,250.0,257.0,0.0,0.0,0.0,0.0,0.0,0.0,...,193.0,0,2.420,194.45200,0.380671,0.383535,0.594373,0.644219,749.000,41.20000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1422,Almuwbber et al. 2018,2018,292.6,0.0,125.4,0.0,0.0,0.0,0.0,0.0,...,188.0,0,11.704,197.01208,0.449761,0.471321,1,0.425415,794.965,65.42960
1423,Almuwbber et al. 2018,2018,250.8,0.0,167.2,0.0,0.0,0.0,0.0,0.0,...,188.0,0,11.704,197.01208,0.449761,0.471321,1,0.426657,811.617,67.86096
1424,Almuwbber et al. 2018,2018,209.0,0.0,209.0,0.0,0.0,0.0,0.0,0.0,...,188.0,0,11.704,197.01208,0.449761,0.471321,1,0.427906,856.056,68.65800
1425,Almuwbber et al. 2018,2018,167.2,0.0,250.8,0.0,0.0,0.0,0.0,0.0,...,188.0,0,11.704,197.01208,0.449761,0.471321,1,0.429035,865.933,67.73456


In [14]:
scc_strength_clean_wo_issue = scc_strength_clean[~index]

In [15]:
df_chem_for_strength = pd.read_csv('df_chem_w_strength', index_col=0)
df_chem_for_strength

,Source,Year,Type,SiO2,Al2O3,Fe2O3,CaO,MgO,Na2O,K2O,SO3,TiO2,P2O5,Specific surface area (m2/kg),Specific gravity
0,Bouzoubaa and Lachemi,2001,OPC,20.30,4.20,3.00,62.00,2.80,0.20,0.90,3.50,0.0,0.0,407.000000,3.17
1,Bouzoubaa and Lachemi,2001,FA,52.40,23.40,4.70,13.40,1.30,3.60,0.60,0.20,0.0,0.0,306.000000,2.08
2,Bui et al.,2002,OPC,21.20,5.34,2.28,63.92,3.91,0.00,0.00,2.32,0.0,0.0,368.519231,3.13
3,Bui et al.,2002,FA,33.90,18.70,6.34,23.40,5.31,4.32,0.41,3.74,0.0,0.0,354.583333,2.78
4,Bui et al.,2002,FA,53.20,26.20,5.60,0.80,0.10,0.00,0.00,0.00,0.0,0.0,354.583333,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
244,Sideris et al.,2018,GGBFS,6.53,2.33,8.69,30.86,4.52,0.33,0.31,0.00,0.0,0.0,427.125000,2.45
245,Sideris et al.,2018,LP,1.80,0.45,0.08,65.70,0.68,0.00,0.00,0.00,0.0,0.0,385.750000,2.65
246,Tavasoli et al.,2018,OPC,21.42,4.52,3.48,63.21,1.48,0.52,0.72,2.36,0.0,0.0,357.000000,3.12
247,Tavasoli et al.,2018,GGBFS,26.00,6.85,2.95,57.50,0.00,0.00,0.00,1.25,0.0,0.0,300.000000,2.43


In [16]:
scc_strength_chem_cartesian_clean_wo_issue = pd.merge(scc_strength_clean_wo_issue, df_chem_for_strength, on=('Source', 'Year'))

In [17]:
mix_column_list = list(scc_strength_clean_wo_issue.columns)

#  ['Source',
#  'Year',
#  'OPC',
#  'FA',
#  'GGBFS',
#  'SF',
#  'MK',
#  'BA',
#  'CKD',
#  'NP',
#  'POFA',
#  'RHA',
#  'VA',
#  'Zeolite',
#  'Pumice',
#  'Coarse aggregate',
#  'Fine aggregate',
#  'Water',
#  'Added Water',
#  'Superplasticizer',
#  'water -eff',
#  'w/b (eff)',
#  'w/b (total)',
#  'b/CA',
#  'b/FA',
#  'Slump flow (mm)',
#  '28-d']


In [18]:
def f(x): # x is a record with index and column (in dataframe format)
    try:
        index = mix_column_list.index(x.loc['Type']) 
        return x.loc[['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O', 'SO3', 'TiO2','P2O5', 'Specific surface area (m2/kg)']] * x[index] 
    except ValueError:
        pass

In [19]:
scc_strength_calc_chem_cartesian_clean_wo_issue = pd.concat([scc_strength_chem_cartesian_clean_wo_issue.drop(['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O', 'SO3', 'TiO2','P2O5', 'Specific surface area (m2/kg)'], axis=1),\
                                                             scc_strength_chem_cartesian_clean_wo_issue.apply(f, axis='columns').fillna(0)], axis=1)

# Index(['Source', 'Year', 'OPC', 'FA', 'GGBFS', 'SF', 'MK', 'BA', 'CKD', 'NP',
#        'POFA', 'RHA', 'VA', 'Zeolite', 'Pumice', 'Coarse aggregate',
#        'Fine aggregate', 'Water', 'Added Water', 'Superplasticizer',
#        'water -eff', 'w/b (eff)', 'w/b (total)', 'b/CA', 'b/FA',
#        'Slump flow (mm)', '28-d', 'Type', 'Specific surface area (m2/kg)',
#        'Specific gravity', 'SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O',
#        'K2O', 'SO3', 'TiO2', 'P2O5'],
#       dtype='object')

In [20]:
def f(x):
    list_chem = ['SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO', 'Na2O', 'K2O', 'SO3', 'TiO2','P2O5', 'Specific surface area (m2/kg)']
    return pd.concat([pd.Series(x.loc[chem], index=[f"{chem}_{x.loc['Type']}"]) for chem in list_chem])
     
scc_strength_calc_chem_individual_cartesian_clean_wo_issue = scc_strength_calc_chem_cartesian_clean_wo_issue.apply(f, axis=1)

# redefine scc_strength_calc_chem_cartesian_clean_wo_issue
scc_strength_calc_chem_cartesian_clean_wo_issue = pd.concat([scc_strength_calc_chem_cartesian_clean_wo_issue, scc_strength_calc_chem_individual_cartesian_clean_wo_issue], axis=1).fillna(0)


In [21]:
grouped = scc_strength_calc_chem_cartesian_clean_wo_issue.groupby(mix_column_list) # <pandas.core.groupby.generic.DataFrameGroupBy object at 0x7fe24862da30>


In [22]:
# agg_function = {'SiO2': np.sum, 'Al2O3': np.sum, 'Fe2O3': np.sum, 'CaO': np.sum, 'MgO': np.sum, 'Na2O': np.sum,\
#                 'K2O': np.sum, 'SO3': np.sum, 'TiO2': np.sum, 'P2O5': np.sum, 'Specific surface area (m2/kg)': np.sum}

agg_function = {i:np.sum for i in scc_strength_calc_chem_cartesian_clean_wo_issue.columns[29:]}

scc_strength_calc_chem_clean_wo_issue = grouped.agg(agg_function).reset_index()

# # rearrange the columns to beatify the table
# scc_strength_calc_chem_clean_wo_issue = scc_strength_calc_chem_clean_wo_issue.loc[:, ['Source', 'Year', 'OPC', 'FA', 'GGBFS', 'SF', 'MK', 'BA', 'CKD', 'NP',
#                                                                                        'POFA', 'RHA', 'VA', 'Zeolite', 'Pumice', 'SiO2', 'Al2O3', 'Fe2O3', 'CaO', 'MgO',
#                                                                                        'Na2O', 'K2O', 'SO3', 'TiO2', 'P2O5', 'Specific surface area (m2/kg)', 'Coarse aggregate',
#                                                                                        'Fine aggregate', 'Water', 'Superplasticizer',
#                                                                                        'water -eff', 'w/b (eff)', 'w/b (total)', 'b/CA', 'b/FA',
#                                                                                        'Slump flow (mm)', '28-d']]

In [23]:
scc_strength_calc_chem_clean_wo_issue = scc_strength_calc_chem_clean_wo_issue.drop(['Source', 'Year'], axis=1)

In [24]:
scc_strength_calc_chem_clean_wo_issue.to_csv('concrete_ml_lab_final_project.csv', index=False)